In [29]:
import arcpy
import glob
import os
# Especifica la ruta a la carpeta que contiene subcarpetas
ruta_carpeta = r"D:\MDA\Cucunuba"

# Utiliza glob para buscar todos los archivos .las de forma recursiva
archivos_las = glob.glob(f'{ruta_carpeta}/**/*.las', recursive=True)

# Imprime la lista de archivos encontrados
for archivo in archivos_las:
    print(archivo)
    dtm=archivo.replace("ground.las", "dtm.tif")
    print(dtm)
    arcpy.conversion.LasDatasetToRaster(
        in_las_dataset=archivo,
        out_raster=dtm,
        value_field="ELEVATION",
        interpolation_type="BINNING MINIMUM NATURAL_NEIGHBOR",
        data_type="FLOAT",
        sampling_type="CELLSIZE",
        sampling_value=0.5,
        z_factor=1
    )


# NO BORRAR Codigo para separar Vectores en el PDF 

In [2]:
import os
from PyPDF2 import PdfReader, PdfWriter

def extract_baselines(input_pdf):
    # Cargar el PDF
    reader = PdfReader(input_pdf)
    total_pages = len(reader.pages)

    # Crear una lista para almacenar los índices de las páginas que comienzan con "Baseline BASE"
    baseline_indices = []
    for i in range(total_pages):
        page = reader.pages[i]
        text = page.extract_text()
        if text and "Baseline 2042" in text:
            baseline_indices.append(i)

    # Añadir el índice de la última página
    baseline_indices.append(total_pages)

    # Crear directorio para guardar los archivos PDF separados
    output_dir = os.path.splitext(input_pdf)[0] + "_separated"
    os.makedirs(output_dir, exist_ok=True)

    # Extraer y guardar cada sección en un archivo PDF separado
    for j in range(len(baseline_indices) - 1):
        writer = PdfWriter()
        start_index = baseline_indices[j]
        end_index = baseline_indices[j + 1]

        for k in range(start_index, end_index):
            writer.add_page(reader.pages[k])

        output_pdf_path = os.path.join(output_dir, f"Baseline_{j + 1}.pdf")
        with open(output_pdf_path, "wb") as output_pdf_file:
            writer.write(output_pdf_file)

    print(f"PDFs separados guardados en el directorio: {output_dir}")

# Usar la función con tu archivo PDF
extract_baselines(r"D:\PROYECTOS 2024\RENE\Pacho 2024-06-20\6.Reportes procesamiento\Pacho_2024-06-20 GNSS Processing Report ROVER.pdf")


PDFs separados guardados en el directorio: D:\PROYECTOS 2024\RENE\Pacho 2024-06-20\6.Reportes procesamiento\Pacho_2024-06-20 GNSS Processing Report ROVER_separated


# Pruebas PPK Mavic

In [3]:
pip install -U pyexiftool

Note: you may need to restart the kernel to use updated packages.


### Collect metadata jpg files

In [2]:
import os
import csv
from exiftool import ExifToolHelper

def extract_metadata_to_csv(folder_path, output_csv):
    # Obtener la lista de todos los archivos JPEG en la carpeta
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg')]

    # Lista para almacenar todos los metadatos
    all_metadata = []

    # Extraer metadatos de cada archivo
    with ExifToolHelper() as et:
        for file in files:
            metadata = et.get_metadata(file)
            for d in metadata:
                cleaned_data = {k.split(":")[-1]: v for k, v in d.items()}
                all_metadata.append(cleaned_data)

    # Obtener todas las claves posibles para asegurarse de que el CSV tiene todas las columnas necesarias
    all_keys = set()
    for data in all_metadata:
        all_keys.update(data.keys())
    all_keys = sorted(all_keys)  # Ordenar las claves para un CSV ordenado

    # Escribir los metadatos en el archivo CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=all_keys)
        writer.writeheader()
        for data in all_metadata:
            writer.writerow(data)

# Ruta a la carpeta que contiene las fotos
folder_path = r"D:\MDA\Lomas de Turillas\DJI_202407020813_003_Crear-ruta-de-zona5"
# Ruta al archivo CSV de salida
output_csv = folder_path+"\metadata_img.csv"

# Llamar a la función para extraer los metadatos y exportarlos a CSV
extract_metadata_to_csv(folder_path, output_csv)


### getting the closest UTC time

In [6]:
import pandas as pd
import datetime

def read_pos_file(pos_file_path):
    """Lee un archivo POS y devuelve un DataFrame con las trayectorias."""
    data = []
    with open(pos_file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith('%'):
                continue  # Saltar líneas de encabezado
            parts = line.split()
            if len(parts) >= 14:  # Asumiendo que hay al menos 16 partes en los datos de trayectoria
                utc_time = parts[0] + ' ' + parts[1]
                # Convertir la cadena de tiempo a un objeto datetime
                utc_time = datetime.datetime.strptime(utc_time, "%Y/%m/%d %H:%M:%S.%f")
                data.append({
                    'utc_time': utc_time,
                    'lat': float(parts[2]),
                    'lon': float(parts[3]),
                    'height': float(parts[4]),
                    'sdn': float(parts[7]),
                    'sde': float(parts[8]),
                    'sdu': float(parts[9]),
                    # Agrega más columnas si es necesario
                })
    return pd.DataFrame(data)

def find_closest_time(photo_time, trajectory_times):
    """Encuentra la hora más cercana en la trayectoria para una hora de foto dada."""
    closest_time = min(trajectory_times, key=lambda x: abs(x - photo_time))
    return closest_time

def process_photos_and_trajectories(photos_csv_path, pos_file_path, output_csv_path):
    """Procesa las fotos y las trayectorias, y guarda las correlaciones en un archivo CSV."""
    # Leer los archivos
    photos_metadata = pd.read_csv(photos_csv_path)
    trajectory_data = read_pos_file(pos_file_path)

    # Asegurarse de que las columnas de tiempo estén en formato datetime
    photos_metadata['UTCAtExposure'] = pd.to_datetime(photos_metadata['UTCAtExposure'], format="%Y:%m:%d %H:%M:%S.%f")
    trajectory_data['utc_time'] = pd.to_datetime(trajectory_data['utc_time'])

    # Crear una lista para almacenar los resultados
    results = []

    # Iterar sobre cada fila en los metadatos de las fotos
    for index, row in photos_metadata.iterrows():
        photo_name = row['FileName']
        photo_time = row['UTCAtExposure']
        
        # Encontrar la fila correspondiente en los datos de trayectoria
        closest_time = find_closest_time(photo_time, trajectory_data['utc_time'])
        trajectory_row = trajectory_data.loc[trajectory_data['utc_time'] == closest_time]

        # Extraer los valores de la trayectoria más cercana
        lat = trajectory_row['lat'].values[0]
        lon = trajectory_row['lon'].values[0]
        height = trajectory_row['height'].values[0]
        sdn = trajectory_row['sdn'].values[0]
        sde = trajectory_row['sde'].values[0]
        sdu = trajectory_row['sdu'].values[0]

        # Guardar los resultados
        results.append({
            'photo_name': photo_name,
            'photo_time': photo_time,
            'closest_trajectory_time': closest_time,
            'lat': lat,
            'lon': lon,
            'height': height,
            'sdn': sdn,
            'sde': sde,
            'sdu': sdu
        })

    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame(results)

    # Guardar los resultados en un nuevo archivo CSV
    results_df.to_csv(output_csv_path, index=False)
    print(f"Archivo '{output_csv_path}' generado con éxito.")

# Rutas a los archivos de entrada y salida
photos_csv_path = r"D:\MDA\Lomas de Turillas\DJI_202407020813_003_Crear-ruta-de-zona5\metadata_img.csv"
pos_file_path = r"D:\MDA\Lomas de Turillas\DJI_202407020813_003_Crear-ruta-de-zona5\DJI_202407020813_003_Crear-ruta-de-zona5_PPKOBS.pos"
output_csv_path = r"D:\MDA\Lomas de Turillas\DJI_202407020813_003_Crear-ruta-de-zona5\prueba_PPK.csv"

# Procesar las fotos y trayectorias, y guardar el archivo de salida
process_photos_and_trajectories(photos_csv_path, pos_file_path, output_csv_path)





Archivo 'D:\MDA\Lomas de Turillas\DJI_202407020813_003_Crear-ruta-de-zona5\prueba_PPK.csv' generado con éxito.


## Code complete V1.0

In [10]:
import os
import csv
from exiftool import ExifToolHelper

def extract_metadata_to_csv(folder_path, output_csv):
    # Obtener la lista de todos los archivos JPEG en la carpeta
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.lower().endswith(('.jpg', '.jpeg'))]

    # Lista para almacenar todos los metadatos
    all_metadata = []

    # Extraer metadatos de cada archivo
    with ExifToolHelper() as et:
        for file in files:
            metadata = et.get_metadata(file)
            for d in metadata:
                cleaned_data = {k.split(":")[-1]: v for k, v in d.items()}
                all_metadata.append(cleaned_data)

    # Obtener todas las claves posibles para asegurarse de que el CSV tiene todas las columnas necesarias
    all_keys = set()
    for data in all_metadata:
        all_keys.update(data.keys())
    all_keys = sorted(all_keys)  # Ordenar las claves para un CSV ordenado

    # Escribir los metadatos en el archivo CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=all_keys)
        writer.writeheader()
        for data in all_metadata:
            writer.writerow(data)

# Ruta a la carpeta que contiene las fotos
folder_path = r"D:\MDA\Lomas de Turillas\DJI_202407020803_002_Crear-ruta-de-zona5"
# Ruta al archivo CSV de salida
output_csv = os.path.join(folder_path, "metadata_img.csv")

# Llamar a la función para extraer los metadatos y exportarlos a CSV
extract_metadata_to_csv(folder_path, output_csv)


""" $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"""
import pandas as pd
import datetime
import os

def find_pos_file(directory):
    """Encuentra el archivo .pos en el directorio dado."""
    for file_name in os.listdir(directory):
        if file_name.endswith('.pos'):
            return os.path.join(directory, file_name)
    raise FileNotFoundError("No se encontró un archivo .pos en el directorio especificado.")

def read_pos_file(pos_file_path):
    """Lee un archivo POS y devuelve un DataFrame con las trayectorias."""
    data = []
    with open(pos_file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[24:]:  # Saltar las primeras 24 líneas de encabezado
            if not line.startswith('%'):  # Saltar líneas de encabezado
                parts = line.split()
                if len(parts) >= 14:  # Asumiendo que hay al menos 14 partes en los datos de trayectoria
                    utc_time = parts[0] + ' ' + parts[1]
                    # Convertir la cadena de tiempo a un objeto datetime
                    utc_time = datetime.datetime.strptime(utc_time, "%Y/%m/%d %H:%M:%S.%f")
                    data.append({
                        'utc_time': utc_time,
                        'lat': float(parts[2]),
                        'lon': float(parts[3]),
                        'height': float(parts[4]),
                        'sdn': float(parts[7]),
                        'sde': float(parts[8]),
                        'sdu': float(parts[9]),
                        # Agrega más columnas si es necesario
                    })
    return pd.DataFrame(data)

def find_closest_time(photo_time, trajectory_times):
    """Encuentra la hora más cercana en la trayectoria para una hora de foto dada."""
    closest_time = min(trajectory_times, key=lambda x: abs(x - photo_time))
    return closest_time

def process_photos_and_trajectories(photos_csv_path, pos_directory, output_csv_path):
    """Procesa las fotos y las trayectorias, y guarda las correlaciones en un archivo CSV."""
    # Encontrar el archivo .pos en el directorio especificado
    pos_file_path = find_pos_file(pos_directory)
    
    # Leer los archivos
    photos_metadata = pd.read_csv(photos_csv_path)
    trajectory_data = read_pos_file(pos_file_path)

    # Asegurarse de que las columnas de tiempo estén en formato datetime
    photos_metadata['UTCAtExposure'] = pd.to_datetime(photos_metadata['UTCAtExposure'], format="%Y:%m:%d %H:%M:%S.%f")
    trajectory_data['utc_time'] = pd.to_datetime(trajectory_data['utc_time'])

    # Crear una lista para almacenar los resultados
    results = []

    # Iterar sobre cada fila en los metadatos de las fotos
    for index, row in photos_metadata.iterrows():
        photo_name = row['FileName']
        photo_time = row['UTCAtExposure']
        
        # Encontrar la fila correspondiente en los datos de trayectoria
        closest_time = find_closest_time(photo_time, trajectory_data['utc_time'])
        trajectory_row = trajectory_data.loc[trajectory_data['utc_time'] == closest_time]

        # Extraer los valores de la trayectoria más cercana
        lat = trajectory_row['lat'].values[0]
        lon = trajectory_row['lon'].values[0]
        height = trajectory_row['height'].values[0]
        sdn = trajectory_row['sdn'].values[0]
        sde = trajectory_row['sde'].values[0]
        sdu = trajectory_row['sdu'].values[0]

        # Guardar los resultados
        results.append({
            'photo_name': photo_name,
            'photo_time': photo_time,
            'closest_trajectory_time': closest_time,
            'lat': lat,
            'lon': lon,
            'height': height,
            'sdn': sdn,
            'sde': sde,
            'sdu': sdu
        })

    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame(results)

    # Guardar los resultados en un nuevo archivo CSV
    results_df.to_csv(output_csv_path, index=False)
    print(f"Archivo '{output_csv_path}' generado con éxito.")

# Rutas a los archivos de entrada y salida
photos_csv_path = output_csv
pos_directory = folder_path
output_csv_path = os.path.join(folder_path, "Photos_PPK.csv")

# Procesar las fotos y trayectorias, y guardar el archivo de salida
process_photos_and_trajectories(photos_csv_path, pos_directory, output_csv_path)


Archivo 'D:\MDA\Lomas de Turillas\DJI_202407020803_002_Crear-ruta-de-zona5\Photos_PPK.csv' generado con éxito.


# Join CSV photos

In [2]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los archivos CSV
carpeta_csv = r"D:\MDA\Resguardo Cota"

# Lista para almacenar los DataFrames
dataframes = []

# Iterar sobre los archivos en la carpeta
for archivo in os.listdir(carpeta_csv):
    if archivo.endswith('.csv'):
        ruta_completa = os.path.join(carpeta_csv, archivo)
        df = pd.read_csv(ruta_completa)
        dataframes.append(df)

# Concatenar todos los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame concatenado en un nuevo archivo CSV
ruta_salida = carpeta_csv+"\Photos_PPK_Join.csv"
df_concatenado.to_csv(ruta_salida, index=False)

print(f'Archivos unidos y guardados en: {ruta_salida}')


Archivos unidos y guardados en: D:\MDA\Resguardo Cota\Photos_PPK_Join.csv


In [7]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(r"C:\Users\NicolasViasus\Documents\Temporales_NV\Consolidado_Agua\Consolidado_Posgress.csv",encoding='Latin1')

# Contar el número de registros (filas) en el DataFrame
numero_registros = len(df)

print(f"El archivo CSV tiene {numero_registros} registros.")


El archivo CSV tiene 781590 registros.


In [2]:
df.columns

Index(['muestra', 'informe', 'cliente', 'programa', 'programa_armonizado',
       'municipio', 'fechamuestreo', 'año', 'fecharecepcion', 'fechareporte',
       'punto', 'caudal', 'lluvia', 'tipo_de_agua', 'tipo_de_muestreo',
       'hora_de_toma', 'nortey', 'estex', 'altura_msnm',
       'parametro_armonizado', 'unidades', 'metodo', 'tipo_de_limite',
       'limite', 'valor', 'id', 'latitud', 'longitud'],
      dtype='object')